# Exercice Agentique Météo avec Mistral AI

## Construction d'un système Agentique pour interroger des données météorologiques

---

### Objectifs pédagogiques

- Comprendre et utiliser les **classes abstraites (ABC)**
- Appliquer le **pattern Strategy** pour gérer différents types de prompts
- Maîtriser les **dataclasses** Python
- Gérer les **secrets et configurations**
- Implémenter un **système agentique complet**
- Écrire des **tests unitaires**

### Architecture du système

```
User Question
    ↓
[AgenticWeather] ← Orchestration
    ↓
[GeoData] → Nominatim ← Lat/Lon
    ↓
[WeatherAPI/Client] → OpenWeatherMap ← Données météo
    ↓
[MistralProvider] → Mistral AI ← Réponse en langage naturel
```

## Configuration préalable

### Installation des dépendances

Exécutez cette commande dans votre terminal :

```bash
uv add mistralai geopy requests python-dotenv pytest
```

### Variables d'environnement

Créez un fichier `.env` à la racine du projet :

```
mistral_api_key=votre_clé_mistral
weather_api_key=votre_clé_openweathermap
```

**Obtenir les clés** :
- Mistral AI : https://console.mistral.ai/
- OpenWeatherMap : https://openweathermap.org/api (gratuit avec inscription)

---

## Partie 1 : Logger (Code fourni)

Créez un fichier `src/logger.py` avec ce contenu :

In [1]:
# src/logger.py
import logging

def setup_logging():
    """Configure le système de logging"""
    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
    )

# Test
setup_logging()
logger = logging.getLogger(__name__)
logger.info("Logger configuré avec succès !")

2026-02-05 23:31:13,514 - __main__ - INFO - Logger configuré avec succès !


In [ ]:
from pathlib import Path

# Création du fichier .env
weather_key = input("Weather API Key: ")
mistral_key = input("Mistral API Key: ")

env_content = f'weather_api_key="{weather_key}"\nmistral_api_key="{mistral_key}"\n'
Path(".env").write_text(env_content)

print("✓ Fichier .env créé")

---

## Partie 2 : Modèles de données (Code fourni)

Créez un fichier `src/models.py` :

In [ ]:
# src/models.py
from dataclasses import dataclass

@dataclass
class UserQuery:
    question: str 
    language: str = 'French'

    def __post_init__(self):
        if self.question == "":
            raise ValueError('Must have a question')

@dataclass
class AgentiqueAnswer:
    response: str
    sources: dict

@dataclass
class Document:
    address: dict
    content: dict
    metadata: dict

# Test
query = UserQuery("Quelle est la météo ?")
logger.info(f"UserQuery créé : {query}")

---

## Partie 3 : API Météo

### Question 1.1 : Analyse du code

**Lisez attentivement ce code et identifiez le problème de conception :**

In [ ]:
# src/weather_api.py (VERSION AVEC PROBLÈME)
import logging
import dotenv
import os
import requests
from dataclasses import dataclass
from geopy.geocoders import Nominatim

dotenv.load_dotenv()
logger = logging.getLogger(__name__)

@dataclass
class GeoData:
    address: str
    useragent: str = "my_geocoder"
    timeout: int = 10
    locator: Nominatim = None

    def __post_init__(self):
        if self.locator is None:
            self.locator = Nominatim(user_agent=self.useragent, timeout=self.timeout)
        if self.locator is None:
            logging.error("Can't connect to geocoding agent")
            raise ConnectionError("Unable to initialize geocoder")

    def get_location(self):
        location = self.locator.geocode(self.address)
        if location is None:
            raise ValueError(f"Address not found: {self.address}")
        return location.latitude, location.longitude


@dataclass
class WeatherAPI:
    """CETTE CLASSE A UN PROBLÈME DE CONCEPTION"""
    lat: float
    lon: float
    weather_api_key: str = None

    def __post_init__(self):
        if self.weather_api_key is None:
            self.weather_api_key = os.getenv('weather_api_key') 
            logging.info('Weather api key defined')
        if self.weather_api_key is None:
            raise ValueError('You need to define a Weather api key')
 
    @property
    def url(self):
        return f"https://api.openweathermap.org/data/2.5/weather?lat={self.lat}&lon={self.lon}&appid={self.weather_api_key}"

    def get_weather(self):
        try:
            response = requests.get(self.url)
            response.raise_for_status()
            return response.json()
        except Exception as e:
            logging.error(f'An exception occurred {e}')
            raise

logger.info("Code chargé - analysez-le avant de continuer")

### Votre analyse

**Identifiez le problème de conception majeur dans la classe `WeatherAPI` :**

**Indices :**
- Que se passe-t-il si vous voulez récupérer la météo pour 10 adresses différentes ?
- Combien d'instances de la classe allez-vous créer ?
- La clé API change-t-elle entre chaque requête ?
- Quel principe SOLID est violé ici ?

**Votre réponse :**

```
Le problème est : ________________________________________________

Principe SOLID violé : ____________________________________________

Conséquences : ____________________________________________________
```

### Question 1.2 : Refactoring

**Proposez une nouvelle architecture qui sépare :**
- La **configuration** (clé API, timeout, etc.)
- L'**exécution** de requêtes météo

**Complétez le code ci-dessous :**

In [ ]:
# src/weather_api.py (VERSION REFACTORISÉE)
import logging
import dotenv
import os
import requests
from dataclasses import dataclass
from typing import Tuple

dotenv.load_dotenv()
logger = logging.getLogger(__name__)

# GeoData reste identique
@dataclass
class GeoData:
    address: str
    useragent: str = "my_geocoder"
    timeout: int = 10
    locator = None

    def __post_init__(self):
        if self.locator is None:
            from geopy.geocoders import Nominatim
            self.locator = Nominatim(user_agent=self.useragent, timeout=self.timeout)

    def get_location(self) -> Tuple[float, float]:
        location = self.locator.geocode(self.address)
        if location is None:
            raise ValueError(f"Address not found: {self.address}")
        return location.latitude, location.longitude


# TODO: Créez une classe WeatherConfig pour la configuration
@dataclass
class WeatherConfig:
    """Configuration pour l'API météo"""
    # À COMPLÉTER
    pass


# TODO: Créez une classe WeatherClient pour exécuter les requêtes
@dataclass
class WeatherClient:
    """Client pour effectuer des requêtes météo (réutilisable)"""
    # À COMPLÉTER
    pass


# Test de votre refactoring
# Décommentez quand vous aurez complété le code
# weather_config = WeatherConfig()
# weather_client = WeatherClient(weather_config)
# logger.info("Refactoring réussi !")

---

## Partie 4 : Client Mistral AI - Pattern Strategy

### Code de base fourni

In [ ]:
# src/mistral_api.py (partie 1 - fournie)
import logging
import os 
import dotenv
from mistralai import Mistral
from dataclasses import dataclass
from abc import ABC, abstractmethod
from typing import Optional

dotenv.load_dotenv()
logger = logging.getLogger(__name__)

@dataclass
class MistralSecret:
    mistral_api_key: str = None

    def __post_init__(self):
        if self.mistral_api_key is None:
            self.mistral_api_key = os.getenv('mistral_api_key')
            if self.mistral_api_key:
                logger.info(f'Mistral Key loaded : len {len(self.mistral_api_key)}')
        if self.mistral_api_key is None:
            logger.error('You must set up mistral api key')
            raise ValueError('Mistral API key is required')


@dataclass
class MistralConfig:
    temperature: float = 0
    max_tokens: int = 500
    model: str = "mistral-medium-latest"

    def __post_init__(self):
        if self.temperature < 0:
            raise ValueError('Temperature must be >=0')
        if self.max_tokens <= 0:
            raise ValueError('Max tokens must be >0')

logger.info("MistralSecret et MistralConfig définis")

### Question 2.1 : Classe abstraite Strategy

**La classe abstraite est fournie. Répondez aux questions :**

In [ ]:
# Classe abstraite (fournie)
class MistralStrategy(ABC):
    """Classe abstraite définissant le contrat pour les différentes stratégies de prompt"""
    config: MistralConfig

    @abstractmethod
    def get_prompt(self):
        """Retourne le prompt système pour cette stratégie"""
        pass

    def get_setup(self):
        """Retourne la configuration Mistral"""
        return {
            "temperature": self.config.temperature,
            "max_tokens": self.config.max_tokens,
            "model": self.config.model
        }

logger.info(" MistralStrategy définie")

**Questions de compréhension :**

a) Pourquoi utilise-t-on `ABC` et `@abstractmethod` ?

```
Réponse : ___________________________________________________________
```

b) Que se passe-t-il si on tente d'instancier directement `MistralStrategy` ?

```
Réponse : ___________________________________________________________
```

**Testez-le :**

In [ ]:
# Test : essayez d'instancier MistralStrategy
# Décommentez et exécutez
# try:
#     strategy = MistralStrategy()
# except TypeError as e:
#     logger.info(f"Erreur attendue : {e}")

### Question 2.2 : Stratégie Agentique (fournie)

In [ ]:
# AgenticStrategy (fournie)
@dataclass
class AgenticStrategy(MistralStrategy):
    content: str
    config: Optional[MistralConfig] = None

    def __post_init__(self):
        if self.content == "":
            raise ValueError("you must enter content for using agentic strategy")

        if self.config is None:
            self.config = MistralConfig(
                temperature=0,
                max_tokens=500,
                model='mistral-medium-latest'
            )

    def get_prompt(self):
        return f"You're an agentic system. You will use the following content: {self.content}. " \
               "If the result is not from the content you must say it clearly."

# Test
test_strategy = AgenticStrategy(content="Test content")
logger.info(f"AgenticStrategy créée")
logger.info(f"Prompt: {test_strategy.get_prompt()[:80]}...")

### Question 2.3 : Implémentez SimpleStrategy

**Créez une stratégie simple sans contexte :**

**Spécifications :**
- Pas de contenu requis
- Prompt système : "You are a helpful assistant. Answer questions clearly and concisely."
- Même configuration par défaut que AgenticStrategy

In [ ]:
# TODO: Implémentez SimpleStrategy
@dataclass
class SimpleStrategy(MistralStrategy):
    """Stratégie pour des questions simples sans contexte spécifique"""
    # À COMPLÉTER
    pass

# Test de votre implémentation
# Décommentez quand terminé
# simple = SimpleStrategy()
# logger.info(f"SimpleStrategy : {simple.get_prompt()}")
# logger.info(f"Config : {simple.get_setup()}")

### Question 2.4 : MistralProvider

**Complétez la méthode `ask_mistral()` :**

In [ ]:
@dataclass
class MistralProvider:
    mistralsecret: MistralSecret
    mistralconfig: MistralConfig

    def __post_init__(self):
        self._client = None

    @property
    def client(self) -> Mistral:
        """Initialise le client Mistral de manière lazy (singleton pattern)"""
        if self._client is None:
            try:
                self._client = Mistral(api_key=self.mistralsecret.mistral_api_key)
                logger.info('Mistral Client initialized')
            except ValueError as e:
                logger.error(f'Erreur: {e}')
                raise ValueError('Mistral key is not defined')
            except Exception as e:
                raise ConnectionError(f'Error {e}')
        return self._client
    
    def _create_message(self, prompt: str, strategy: MistralStrategy) -> list[dict]:
        """Crée la liste de messages pour l'API Mistral"""
        return [
            {'role': 'system', "content": strategy.get_prompt()},
            {'role': 'user', "content": prompt}
        ]
    
    def ask_mistral(self, prompt: str, context: str = "", strategy: Optional[MistralStrategy] = None):
        """
        Envoie une requête à Mistral AI
        
        Args:
            prompt: La question de l'utilisateur
            context: Le contexte pour l'agentique (optionnel)
            strategy: La stratégie à utiliser (si None, utilise AgenticStrategy avec le context)
        
        Returns:
            La réponse de Mistral
        """
        # TODO: Complétez cette méthode
        # 1. Si strategy est None et context n'est pas vide, créer une AgenticStrategy
        # 2. Si strategy est None et context est vide, créer une SimpleStrategy
        # 3. Logger la stratégie utilisée
        # 4. Créer les messages
        # 5. Récupérer la config
        # 6. Appeler l'API Mistral avec client.chat.complete()
        # 7. Gérer les erreurs
        
        pass

logger.info("MistralProvider défini - complétez ask_mistral()")

---

## Partie 5 : Système Agentique Météo

**Utilisez votre WeatherClient refactorisé dans AgenticWeather :**

In [ ]:
@dataclass
class AgenticWeather:
    """Système agentique pour répondre à des questions sur la météo"""
    mistralprovider: MistralProvider
    weather_client: WeatherClient  # Utilise votre refactoring !

    def _get_context(self, address: str) -> dict:
        """Récupère les données météo pour une adresse donnée"""
        # TODO: Complétez en utilisant votre WeatherClient
        pass
    
    def ask_weather_question(self, address: str, question: str) -> str:
        """Pose une question sur la météo d'une adresse"""
        weather_data = self._get_context(address)
        context_text = self._format_weather_data(weather_data)
        response = self.mistralprovider.ask_mistral(question, context_text)
        return response.choices[0].message.content
    
    def _format_weather_data(self, data: dict) -> str:
        """Formate les données JSON de l'API météo en texte lisible"""
        try:
            weather = data['weather'][0]
            main = data['main']
            wind = data['wind']
            
            temp_celsius = main['temp'] - 273.15
            feels_like_celsius = main['feels_like'] - 273.15
            
            context = f"""
Données météorologiques pour {data['name']}:
- Conditions: {weather['description']}
- Température: {temp_celsius:.1f}°C (ressenti: {feels_like_celsius:.1f}°C)
- Humidité: {main['humidity']}%
- Pression: {main['pressure']} hPa
- Vent: {wind['speed']} m/s
- Visibilité: {data.get('visibility', 'N/A')} mètres
            """.strip()
            return context
        except KeyError as e:
            logger.error(f'Error formatting weather data: {e}')
            return str(data)

logger.info("AgenticWeather défini")

---

## Partie 6 : Tests

**Écrivez des tests pour vérifier votre code :**

In [ ]:
# Tests pour les stratégies
import pytest

def test_agentic_strategy_with_empty_content():
    """Test que AgenticStrategy lève une erreur si content est vide"""
    # TODO: Écrivez le test
    pass

def test_simple_strategy_initialization():
    """Test l'initialisation de SimpleStrategy"""
    # TODO: Écrivez le test
    pass

# Exécutez les tests
# pytest.main(['-v', __file__])

---

## Partie 7 : Programme principal

**Créez un programme qui utilise tout le système :**

In [ ]:
# Programme principal
def main():
    """Point d'entrée du programme"""
    try:
        # TODO: Initialisez tous les composants
        # 1. MistralSecret et MistralConfig
        # 2. MistralProvider
        # 3. WeatherConfig et WeatherClient
        # 4. AgenticWeather
        
        # TODO: Demandez à l'utilisateur une adresse et une question
        
        # TODO: Affichez la réponse
        
        pass
        
    except ValueError as e:
        print(f"Erreur de configuration : {e}")
    except Exception as e:
        print(f"Erreur : {e}")

# Décommentez pour tester
# main()

---

## Récapitulatif

### Ce que vous avez appris :

✅ **Classes abstraites (ABC)** : Définir des contrats pour les stratégies

✅ **Pattern Strategy** : Différents comportements de prompts interchangeables

✅ **Séparation des responsabilités** : Config vs Exécution

✅ **Injection de dépendances** : Clients réutilisables

✅ **Gestion d'erreurs** : Try/except appropriés

✅ **Tests unitaires** : Vérification du code

### Principes SOLID appliqués :

- **S** : Single Responsibility
- **O** : Open/Closed (extensible via nouvelles stratégies)
- **L** : Liskov Substitution (stratégies interchangeables)
- **D** : Dependency Inversion (dépendance sur ABC)

---

## Critères d'évaluation

- **Architecture** : Refactoring WeatherAPI, utilisation correcte des ABC
- **Code quality** : Conventions Python, logging, gestion d'erreurs
- **Implémentation** : SimpleStrategy, ask_mistral(), AgenticWeather
- **Tests** : Tests unitaires fonctionnels

Bonne chance !